In [37]:
import requests

from config import SPOT_CLIENT_SECRET, SPOT_CLIENT_ID
CLIENT_ID = SPOT_CLIENT_ID
CLIENT_SECRET = SPOT_CLIENT_SECRET


def get_spotify_token(client_id, client_secret):
    auth_response = requests.post(
        'https://accounts.spotify.com/api/token',
        {'grant_type': 'client_credentials'},
        auth=(client_id, client_secret)
    )
    return auth_response.json()['access_token']

def search_track(track_name, token):
    headers = {'Authorization': f'Bearer {token}'}
    search_params = {
        'q': track_name,
        'type': 'track',
        'limit': 1
    }
    response = requests.get('https://api.spotify.com/v1/search', headers=headers, params=search_params)
    return response.json()

# credenciais do Spotify
client_id = CLIENT_ID
client_secret = CLIENT_SECRET

# Obter o token
token = get_spotify_token(client_id, client_secret)

# Buscar uma música
track_name = 'Heaven Bryan Adams'
track_info = search_track(track_name, token)


In [38]:
def get_track_features(track_id, token):
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(f'https://api.spotify.com/v1/audio-features/{track_id}', headers=headers)
    return response.json()

#ID da música do passo anterior
track_id = track_info['tracks']['items'][0]['id']
features = get_track_features(track_id, token)


In [39]:
print(features)

#detalhes da faixa a partir da API
track_details = search_track(track_name, token)

# nome e o artista corretamente
track_info = {
    'name': track_details['tracks']['items'][0]['name'],
    'artist': track_details['tracks']['items'][0]['artists'][0]['name'],
    # outras informações conforme necessário
}


{'danceability': 0.382, 'energy': 0.589, 'key': 9, 'loudness': -7.315, 'mode': 0, 'speechiness': 0.0267, 'acousticness': 0.0465, 'instrumentalness': 0, 'liveness': 0.122, 'valence': 0.335, 'tempo': 140.059, 'type': 'audio_features', 'id': '7Ewz6bJ97vUqk5HdkvguFQ', 'uri': 'spotify:track:7Ewz6bJ97vUqk5HdkvguFQ', 'track_href': 'https://api.spotify.com/v1/tracks/7Ewz6bJ97vUqk5HdkvguFQ', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7Ewz6bJ97vUqk5HdkvguFQ', 'duration_ms': 243360, 'time_signature': 4}


In [40]:
# modelo carregado
import pandas as pd
import joblib

from tensorflow.keras.models import load_model

##neural_model_loaded = load_model('nn_model.keras')
model_loaded = joblib.load('melhor_knn_model.joblib')

# Preparar os dados

import pandas as pd
from sklearn.preprocessing import StandardScaler

# 'features' é um dicionário com as características extraídas da API do Spotify
new_data = pd.DataFrame([features])

# Selecionar apenas as colunas relevantes
relevant_columns = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
new_data = new_data[relevant_columns]

# Aplicando a normalização
scaler = StandardScaler()
new_data[relevant_columns] = scaler.fit_transform(new_data[relevant_columns])

# Aplicar a mesma normalização/padronização usada no treinamento
new_data_scaled = scaler.transform(new_data)

# Fazer a previsão
quadrant_prediction = model_loaded.predict(new_data_scaled)


c:\Users\luboh\anaconda3\envs\spotenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\luboh\anaconda3\envs\spotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\luboh\anaconda3\envs\spotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\luboh\anaconda3\envs\spotenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Ch

In [41]:


quadrants = {
    0: 'Alta Valência, Alta Ativação (Alegre, Energética)',
    1: 'Baixa Valência, Alta Ativação (Tensa, Nervosa)',
    2: 'Baixa Valência, Baixa Ativação (Triste, Melancólica)',
    3: 'Alta Valência, Baixa Ativação (Calma, Relaxante)',
 'Q1': 'Alta Valência, Alta Ativação (Alegre, Energética)',
 'Q2': 'Baixa Valência, Alta Ativação (Tensa, Nervosa)',
 'Q3': 'Baixa Valência, Baixa Ativação (Triste, Melancólica)',
 'Q4': 'Alta Valência, Baixa Ativação (Calma, Relaxante)'
}

# 'quadrants' é uma lista ou dicionário que mapeia os números da previsão para os rótulos dos quadrantes
predicted_quadrant = quadrants[quadrant_prediction[0]]

# Combinando as informações
track_info['predicted_quadrant'] = predicted_quadrant

# Exibindo as informações
print(f"Nome da Música: {track_info['name']}")
print(f"Artista: {track_info['artist']}")
print(f"Quadrante Previsto: {track_info['predicted_quadrant']}")
# ... exibir outras informações desejadas


Nome da Música: Heaven
Artista: Bryan Adams
Quadrante Previsto: Baixa Valência, Baixa Ativação (Triste, Melancólica)


In [8]:
import numpy as np

# Dicionário de mapeamento dos números da previsão para os rótulos dos quadrantes
quadrants = {
    0: 'Alta Valência, Alta Ativação (Alegre, Energética)',
    1: 'Baixa Valência, Alta Ativação (Tensa, Nervosa)',
    2: 'Baixa Valência, Baixa Ativação (Triste, Melancólica)',
    3: 'Alta Valência, Baixa Ativação (Calma, Relaxante)'
}

# Supondo que `quadrant_prediction` é o resultado da rede neural
# Aqui você deve ter o resultado da chamada `model.predict(...)`
# quadrant_prediction = model.predict(some_input_data)

# Obtenha o índice da classe com a maior probabilidade para a previsão
predicted_class_index = np.argmax(quadrant_prediction, axis=-1)

# Agora você pode acessar o dicionário usando esse índice
# Note que `predicted_class_index` é um array, então você acessa o primeiro elemento com [0]
predicted_quadrant = quadrants[predicted_class_index[0]]

# Supondo que 'track_info' é um dicionário contendo informações da música
# track_info = {'name': 'Name of the Song', 'artist': 'Name of the Artist', ...}

# Adicione o quadrante previsto ao dicionário de informações da música
track_info['predicted_quadrant'] = predicted_quadrant

# Exiba as informações
print(f"Nome da Música: {track_info['name']}")
print(f"Artista: {track_info['artist']}")
print(f"Quadrante Previsto: {track_info['predicted_quadrant']}")
# ... exibir outras informações desejadas


IndexError: invalid index to scalar variable.